In [1]:
import sys

sys.path.append("../")

In [2]:
import cheetah
import matplotlib.pyplot as plt
import numpy as np
from gymnasium.wrappers import RescaleAction
from stable_baselines3.common.env_checker import check_env

from src.environments.awake_e_steering import AwakeESteering
from compare.cern_shared.environment import e_trajectory_simENV

In [3]:
old_env = e_trajectory_simENV()
new_env = AwakeESteering()
wrapped_new_env = RescaleAction(new_env, -1, 1)

/Users/jankaiser/Documents/DESY/rl4aa24-tutorial/notebooks/../compare/cern_shared/utils/twissReader.py:308: UserWarning: MISSING FIELD PX IN TWISS INPUT ../src/environments/electron_tt43.out
  warnings.warn(
/Users/jankaiser/Documents/DESY/rl4aa24-tutorial/notebooks/../compare/cern_shared/utils/twissReader.py:331: UserWarning: MISSING FIELD PY IN TWISS INPUT../src/environments/electron_tt43.out
  warnings.warn(


In [4]:
check_env(old_env)

In [5]:
check_env(wrapped_new_env)

In [7]:
old_env.reset()

(array([-0.01408744, -0.01990961, -0.05529934, -0.0971205 , -0.17207842,
         0.22096458,  0.4636218 ,  0.3623196 ,  0.23874807,  0.18251742],
       dtype=float32),
 {})

In [8]:
new_env.reset()

(array([ 3.5436096e-05, -4.8860282e-05,  1.9524482e-04,  6.3794572e-04,
         1.4513392e-03, -2.0595023e-03, -4.5622676e-03, -3.6462608e-03,
        -2.4693348e-03, -1.9142326e-03], dtype=float32),
 {'backend_info': {'incoming_beam': array([1.0000000e+08, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
          0.0000000e+00, 9.8234415e-04, 1.9646883e-04, 9.8234415e-04,
          1.9646883e-04, 1.0000000e-06, 1.0000000e-06], dtype=float32),
   'quadrupole_settings': array([ -5.085799,   9.504011,  -8.240125, -31.725054,  19.703522,
          -31.725054,  -7.92377 ,  26.87176 ,  -1.07907 ,  16.7789  ,
          -14.029   ], dtype=float32)}})